# ETL of Dow 30 Stock Data
#### Fabienne Zumbuehl; James Ye; Tanvir Khan

In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
from sqlalchemy import create_engine

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## price data from cnbc.com

In [3]:
price_url = "https://www.cnbc.com/dow-30/"

In [4]:
browser.visit(price_url)
time.sleep(5)
price_html = browser.html

In [5]:
price_data_tables = pd.read_html(price_html)
print("Number of tables: ", len(price_data_tables))
price_data_df = price_data_tables[0]
#price_data_df.set_index('SYMBOL', inplace=True)

# drop ticker 'Dow' because their substainability can not be found in Yahoo Finance
price_data_df.drop(index=29, inplace=True)



Number of tables:  1


In [6]:
price_data_df.index.name = 'id'
price_data_df

,SYMBOL,NAME,PRICE,CHANGE,%CHANGE,LOW,HIGH,PREVIOUS CLOSE
id,,,,,,,,
0,AXP,American Express Co,89.83,-0.90,-0.992,89.125,91.430,90.73
1,AAPL,Apple Inc,316.85,-2.38,-0.746,315.870,320.890,319.23
2,BA,Boeing Co,139.00,5.68,4.260,136.151,144.239,133.32
3,CAT,Caterpillar Inc,114.06,-1.63,-1.409,113.820,115.680,115.69
4,CSCO,Cisco Systems Inc,44.64,-0.80,-1.761,44.555,45.655,45.44
5,CVX,Chevron Corp,92.04,-0.96,-1.032,91.280,93.370,93.00
6,XOM,Exxon Mobil Corp,44.56,-0.81,-1.785,44.390,45.790,45.37
7,GS,Goldman Sachs Group Inc,180.10,-1.34,-0.739,177.680,181.170,181.44
8,HD,Home Depot Inc,240.88,2.69,1.130,235.790,241.180,238.19


In [7]:
dow29_symbols = price_data_df['SYMBOL']
dow29_symbols

id
0      AXP
1     AAPL
2       BA
3      CAT
4     CSCO
5      CVX
6      XOM
7       GS
8       HD
9      IBM
10    INTC
11     JNJ
12      KO
13     JPM
14     MCD
15     MMM
16     MRK
17    MSFT
18     NKE
19     PFE
20      PG
21     TRV
22     UNH
23     RTX
24      VZ
25       V
26     WBA
27     WMT
28     DIS
Name: SYMBOL, dtype: object

## dividend data from dividend.com

In [8]:
dividend_url = "https://www.dividend.com/dividend-stocks/dow-30-dividend-stocks/"

In [9]:
dividend_data_tables = pd.read_html(dividend_url)
print("Number of tables: ", len(dividend_data_tables))
dividend_data_df = dividend_data_tables[0]
dividend_data_df.head()

Number of tables:  1


,Unnamed: 0,Stock Symbol,Company Name,DARSâ¢ Rating,Dividend Yield,Closing Price,Annualized Dividend,Ex-Div Date,Pay Date
0,NaN,XOM,Exxon Mobil,locked,7.87%,$45.74,$3.4800,2020-05-12,2020-06-10
1,NaN,BA,Boeing Co.,locked,6.16%,$128.91,$8.2200,2020-02-13,2020-03-06
2,NaN,CVX,Chevron Corp,locked,5.40%,$93.37,$5.1600,2020-05-18,2020-06-10
3,NaN,IBM,IBM Corp,locked,5.30%,$122.59,$6.5200,2020-05-07,2020-06-10
4,NaN,UTX,United Technologies,locked,5.16%,$86.01,$2.9400,2020-02-13,2020-03-10


In [10]:
# only run these once!!!
del(dividend_data_df["Unnamed: 0"])
#dividend_data_df.set_index("Stock Symbol", inplace=True)
dividend_data_df.head()

,Stock Symbol,Company Name,DARSâ¢ Rating,Dividend Yield,Closing Price,Annualized Dividend,Ex-Div Date,Pay Date
0,XOM,Exxon Mobil,locked,7.87%,$45.74,$3.4800,2020-05-12,2020-06-10
1,BA,Boeing Co.,locked,6.16%,$128.91,$8.2200,2020-02-13,2020-03-06
2,CVX,Chevron Corp,locked,5.40%,$93.37,$5.1600,2020-05-18,2020-06-10
3,IBM,IBM Corp,locked,5.30%,$122.59,$6.5200,2020-05-07,2020-06-10
4,UTX,United Technologies,locked,5.16%,$86.01,$2.9400,2020-02-13,2020-03-10


In [11]:
dividend_reduced_df = dividend_data_df[['Dividend Yield', 'Annualized Dividend', 'Ex-Div Date', 'Pay Date']].copy()
#dividend_reduced_df.index.name = "SYMBOL"
#dividend_reduced_df.drop(index=29, inplace=True)
dividend_df = dividend_reduced_df
dividend_df.index.name = 'id'
dividend_df

,Dividend Yield,Annualized Dividend,Ex-Div Date,Pay Date
id,,,,
0,7.87%,$3.4800,2020-05-12,2020-06-10
1,6.16%,$8.2200,2020-02-13,2020-03-06
2,5.40%,$5.1600,2020-05-18,2020-06-10
3,5.30%,$6.5200,2020-05-07,2020-06-10
4,5.16%,$2.9400,2020-02-13,2020-03-10
5,4.50%,$1.8300,2020-05-19,2020-06-12
6,4.11%,$1.5200,2020-05-07,2020-06-05
7,4.03%,$5.8800,2020-02-13,2020-03-12
8,3.95%,$3.6000,2020-04-03,2020-04-30


In [12]:
dividend_df['Dividend Yield'] = dividend_df['Dividend Yield'].str.replace('%','').astype(float)
dividend_df['Annualized Dividend'] = dividend_df['Annualized Dividend'].str.replace('$','').astype(float)


In [13]:
print(dividend_df.dtypes)
dividend_df.head()

Dividend Yield         float64
Annualized Dividend    float64
Ex-Div Date             object
Pay Date                object
dtype: object


,Dividend Yield,Annualized Dividend,Ex-Div Date,Pay Date
id,,,,
0,7.87,3.48,2020-05-12,2020-06-10
1,6.16,8.22,2020-02-13,2020-03-06
2,5.40,5.16,2020-05-18,2020-06-10
3,5.30,6.52,2020-05-07,2020-06-10
4,5.16,2.94,2020-02-13,2020-03-10


## finance.yahoo.com

In [14]:
print("ticker", "ESG Score")
egs_rating_list = []
first = True
for stock_symbol in dow29_symbols:
    sustainability_url = f"https://finance.yahoo.com/quote/{stock_symbol}/sustainability?p={stock_symbol}"
    browser.visit(sustainability_url)
    if first:
        time.sleep(5)
        first = False
    else:
        time.sleep(1)
    ESG_Risk_Score = browser.find_by_css('div[class="Fz(36px) Fw(600) D(ib) Mend(5px)"]').value
    print(stock_symbol, ESG_Risk_Score)
    egs_rating_list.append(ESG_Risk_Score)

ticker ESG Score
AXP 22
AAPL 24
BA 39
CAT 38
CSCO 14
CVX 40
XOM 34
GS 32
HD 13
IBM 18
INTC 16
JNJ 35
KO 26
JPM 22
MCD 25
MMM 34
MRK 28
MSFT 16
NKE 17
PFE 33
PG 25
TRV 24
UNH 21
RTX 29
VZ 20
V 18
WBA 17
WMT 29
DIS 15


In [15]:
egs_rating_list

['22',
 '24',
 '39',
 '38',
 '14',
 '40',
 '34',
 '32',
 '13',
 '18',
 '16',
 '35',
 '26',
 '22',
 '25',
 '34',
 '28',
 '16',
 '17',
 '33',
 '25',
 '24',
 '21',
 '29',
 '20',
 '18',
 '17',
 '29',
 '15']

In [16]:
price_data_df['ESG Rating'] = egs_rating_list


In [17]:
price_data_df['ESG Rating'] = pd.to_numeric(price_data_df['ESG Rating'])
price_data_df.rename(columns={'%CHANGE':'PCT_CHANGE'}, inplace=True)

#price_data_df['NAME'] = price_data_df['NAME'].astype(str)
price_data_df.dtypes

SYMBOL             object
NAME               object
PRICE             float64
CHANGE            float64
PCT_CHANGE        float64
LOW               float64
HIGH              float64
PREVIOUS CLOSE    float64
ESG Rating          int64
dtype: object

In [26]:
price_data_df

,SYMBOL,NAME,PRICE,CHANGE,PCT_CHANGE,LOW,HIGH,PREVIOUS CLOSE,ESG Rating
id,,,,,,,,,
0,AXP,American Express Co,89.83,-0.90,-0.992,89.125,91.430,90.73,22
1,AAPL,Apple Inc,316.85,-2.38,-0.746,315.870,320.890,319.23,24
2,BA,Boeing Co,139.00,5.68,4.260,136.151,144.239,133.32,39
3,CAT,Caterpillar Inc,114.06,-1.63,-1.409,113.820,115.680,115.69,38
4,CSCO,Cisco Systems Inc,44.64,-0.80,-1.761,44.555,45.655,45.44,14
5,CVX,Chevron Corp,92.04,-0.96,-1.032,91.280,93.370,93.00,40
6,XOM,Exxon Mobil Corp,44.56,-0.81,-1.785,44.390,45.790,45.37,34
7,GS,Goldman Sachs Group Inc,180.10,-1.34,-0.739,177.680,181.170,181.44,32
8,HD,Home Depot Inc,240.88,2.69,1.130,235.790,241.180,238.19,13


# load data into database

In [21]:
connection_string = "postgres:postgres@localhost:5433/stock_db"
engine = create_engine(f'postgresql://{connection_string}')

In [22]:
# Confirm tables
engine.table_names()

['price_data_df', 'dividend_df']

In [28]:
price_data_df.head()

,SYMBOL,NAME,PRICE,CHANGE,PCT_CHANGE,LOW,HIGH,PREVIOUS CLOSE,ESG Rating
id,,,,,,,,,
0,AXP,American Express Co,89.83,-0.90,-0.992,89.125,91.430,90.73,22
1,AAPL,Apple Inc,316.85,-2.38,-0.746,315.870,320.890,319.23,24
2,BA,Boeing Co,139.00,5.68,4.260,136.151,144.239,133.32,39
3,CAT,Caterpillar Inc,114.06,-1.63,-1.409,113.820,115.680,115.69,38
4,CSCO,Cisco Systems Inc,44.64,-0.80,-1.761,44.555,45.655,45.44,14


In [25]:
price_data_df.to_sql(name='price_data_df', con=engine, if_exists='append', index=True)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "SYMBOL" of relation "price_data_df" does not exist
LINE 1: INSERT INTO price_data_df (id, "SYMBOL", "NAME", "PRICE", "C...
                                       ^

[SQL: INSERT INTO price_data_df (id, "SYMBOL", "NAME", "PRICE", "CHANGE", "PCT_CHANGE", "LOW", "HIGH", "PREVIOUS CLOSE", "ESG Rating") VALUES (%(id)s, %(SYMBOL)s, %(NAME)s, %(PRICE)s, %(CHANGE)s, %(PCT_CHANGE)s, %(LOW)s, %(HIGH)s, %(PREVIOUS CLOSE)s, %(ESG Rating)s)]
[parameters: ({'id': 0, 'SYMBOL': 'AXP', 'NAME': 'American Express Co', 'PRICE': 89.83, 'CHANGE': -0.9, 'PCT_CHANGE': -0.992, 'LOW': 89.125, 'HIGH': 91.43, 'PREVIOUS CLOSE': 90.73, 'ESG Rating': 22}, {'id': 1, 'SYMBOL': 'AAPL', 'NAME': 'Apple Inc', 'PRICE': 316.85, 'CHANGE': -2.38, 'PCT_CHANGE': -0.746, 'LOW': 315.87, 'HIGH': 320.89, 'PREVIOUS CLOSE': 319.23, 'ESG Rating': 24}, {'id': 2, 'SYMBOL': 'BA', 'NAME': 'Boeing Co', 'PRICE': 139.0, 'CHANGE': 5.68, 'PCT_CHANGE': 4.26, 'LOW': 136.151, 'HIGH': 144